In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('../input/chapter-2-carpricedata/data.csv')
df


In [ ]:
# Makes reading the column headings much easier
df.head().T

In [ ]:
df_hw = df[['Make', 'Model', 'Year', 'Engine HP', 'Engine Cylinders', 'Transmission Type', 'Vehicle Style', 'highway MPG', 'city mpg', 'MSRP']]
df_hw.columns

In [ ]:
df_hw.columns = df_hw.columns.str.lower().str.replace(' ', '_')
df_hw.columns

In [ ]:
# Rename MSRP value to price
# 
# Could also be done with pd.rename
df_hw = df_hw.rename(columns={'msrp': 'price'})
# 
# 


In [ ]:
df_hw.isnull().sum()
# Fill in empty values for engine_hp and engine_cylinders as 0

In [ ]:
df_hw = df_hw.fillna(0)
df_hw.isnull().sum()

In [ ]:
#  What is the most frequent observation (mode) for the column transmission_type?
df_hw.transmission_type.value_counts()
# Automatic

In [ ]:
df_hw.dtypes

In [ ]:
categorical_columns = list(df_hw.dtypes[df_hw.dtypes  == 'object'].index)
categorical_columns

In [ ]:
numerical_columns = list(df_hw.dtypes[(df_hw.dtypes == 'int64') | (df_hw.dtypes == 'float64')].index)
print(numerical_columns)
del numerical_columns[-1]
print(numerical_columns)



In [ ]:
from IPython.display import display    
df_hw = df_hw.reset_index(drop=True)
df_hw
if isinstance(df_hw, pd.DataFrame):
  print('df is a DataFrame')
else:
  print('df is not a DataFrame')

In [ ]:
# df_hw[numerical_columns].corrwith(df_hw['year'])

In [ ]:
# Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, 
# you compute the correlation coefficient between every pair of features in the dataset.
for col in numerical_columns:
    print(col)
    display(df_hw[numerical_columns].corrwith(df_hw[col]))
    print()
    print()
# highwaympg and city_mpg

In [ ]:
price_mean = df_hw.price.mean()
price_mean

In [ ]:
# Let's create a variable above_average which is 1 if the price is above its mean value and 0 otherwise.

price = (df_hw['price'] >= price_mean)
above_average = price.astype(int)
df_hw['price'].head(), above_average.head(), price.head()

In [ ]:
# Split your data in train/val/test sets with 60%/20%/20% distribution.
from sklearn.model_selection import train_test_split

In [ ]:
# set the seed as 42
df_full_train, df_test = train_test_split(df_hw, test_size=0.2, random_state=42)
len(df_full_train)

In [ ]:
df_train, df_val = train_test_split(df_full_train, test_size=.25, random_state=42)
len(df_train), len(df_val), len(df_test)
print(df_hw.columns, df_train.columns)

In [ ]:
# get y_values
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train

In [ ]:
# get above_average for each dataset
price_mean = df_hw['price'].mean()
print(price_mean)
def get_above_average(df):
    price = (df['price'] >= price_mean)
    return price.astype(int)

In [ ]:
train_above_average = get_above_average(df_train)
val_above_average = get_above_average(df_val)
test_above_average = get_above_average(df_test)
train_above_average[:20], df_train.price[:20]

In [ ]:
y_train = train_above_average
y_test = val_above_average
y_val = test_above_average


In [ ]:
del df_train['price']
del df_val['price']
del df_test['price']

In [ ]:
df_train.columns

In [ ]:
from sklearn.metrics import mutual_info_score


In [ ]:
# Calculate the Mutual Information Score between Above_Average 
for c in categorical_columns:
    print(c)
    score = mutual_info_score(df_train[c], train_above_average)
    display(score)
    print()

#     Transmission Type

In [ ]:
# Train Logistic Regression
# 
# First, we do one hot encoding on the Categorical values
from sklearn.feature_extraction import DictVectorizer

In [ ]:
train_dicts = df_train.to_dict(orient="records")
val_dicts = df_val.to_dict(orient="records")
test_dicts = df_test.to_dict(orient="records")



In [ ]:
dv = DictVectorizer(sparse=False)

In [ ]:
X_train = dv.fit_transform(train_dicts)
X_train.shape

In [ ]:
# There is no need to initialize another instance of dictvectorizer after fitting it on the train set as it will overwrite what it learnt from being fit on the train data.
# 
# The correct way is to fit_transform the train set, and only transform the validation and test sets.	
# 
X_val = dv.transform(val_dicts)
X_val.shape

In [ ]:
X_test = dv.transform(test_dicts)
X_test.shape

In [ ]:
# dv.get_feature_names_out()
# gives us a very long list

In [ ]:
# Training the Logistic Regression
from sklearn.linear_model import LogisticRegression


In [ ]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# The weights
w = model.coef_[0].round(2)
w

In [ ]:
w0 = model.intercept_[0]
w0

In [ ]:
model.predict(X_train)

In [ ]:
model.predict_proba(X_train)
y_pred = model.predict_proba(X_val)[:, 1].round(2)
y_pred[:10]

In [ ]:
price_decision = (y_pred >= 0.5)
price_decision


In [ ]:
df_val[price_decision][['model', 'make']]

In [ ]:
round((y_val == price_decision.astype(int)).mean(), 2)
# 0.61 accuracy

In [ ]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(3)))
